In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [688]:
import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, VGG19, Xception, InceptionV3


In [689]:
train_data_dir = "../data/processed/train"
val_data_dir = "../data/processed/val"

In [690]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [691]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(400,400),
    batch_size=4,
    class_mode="categorical"
)

Found 21 images belonging to 2 classes.


In [692]:
val_generator = val_datagen.flow_from_directory(
    train_data_dir,
    target_size = (400,400),
    batch_size = 4,
    class_mode = "categorical"
)

Found 21 images belonging to 2 classes.


In [696]:
vgg16 = tf.keras.applications.mobilenet.MobileNet(
    include_top=False,
    weights="imagenet",
    input_shape=(400,400,3),
    name="vgg16",
)


/tmp/ipykernel_109284/867091459.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  vgg16 = tf.keras.applications.mobilenet.MobileNet(


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


In [697]:
for layer in vgg16.layers:
    layer.trainable = False

In [698]:
classesNum = len(os.listdir(train_data_dir))
classesNum

2

In [699]:
data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip("horizontal"),
                                         tf.keras.layers.RandomRotation(0.2), 
                                         tf.keras.layers.RandomZoom(0.2),
                                         tf.keras.layers.RandomHeight(0.2),
                                         tf.keras.layers.RandomWidth(0.2),],
                                         name ="data_augmentation")

In [700]:
model = Sequential()
#model.add(tf.keras.Input(shape=(224, 224, 3)))
#model.add(data_augmentation)
model.add(vgg16)
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dense(classesNum, activation="softmax"))

print(model.summary())


Model: "sequential_39"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 12, 12, 1024)   │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_132         │ (None, 12, 12, 1024)   │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_21 (Flatten)            │ (None, 147456)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_67 (Dense)                │ (None, 1024)           │   150,995,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_68 (Dense)                │ (None, 2)              │         2,050 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 154,230,978 (588.34 MB)

 Trainable params: 151,000,066 (576.02 MB)

 Non-trainable params: 3,230,912 (12.32 MB)

None


In [701]:
model.compile(
    optimizer="adam", 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [702]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // train_generator.batch_size,
    validation_data = val_generator,
    validation_steps = val_generator.samples // val_generator.batch_size,
    epochs=10,
    verbose=1
)

Epoch 1/10


/home/serhan/Documents/GitHub/human-segmentation/venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.5099 - loss: 16.8489 - val_accuracy: 0.7000 - val_loss: 18.8491
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5000 - loss: 119.8326 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10


/usr/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9676 - loss: 0.2937 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10


2024-09-07 18:31:42.602042: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9700 - loss: 0.2502 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [703]:
def preprocess_image(image_path, image_size=(400,400)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

In [729]:
image_path = "nis3.jpg"

preprocessed_image = preprocess_image(image_path)

In [730]:
predictions = model.predict(preprocessed_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step


In [731]:
predicted_class_index = np.argmax(predictions)  # Get the index of the class with the highest probability
confidence = np.max(predictions)

In [732]:
class_labels = os.listdir("../data/raw")
predicted_class_label = class_labels[predicted_class_index]

print(f"Predicted class: {predicted_class_label} with confidence: {confidence}")

Predicted class: Nisa. with confidence: 1.0


In [728]:
model.save('../models/saved_model/model.h5')